# Effect of repealing Child Tax Credit with Child Dividend

This identifies beneficiaries of the Child Tax Credit by modeling its repeal. Both repeal from current (2017) state and TCJA state are considered on a static basis. Change to after-tax income by decile and share of after-tax income held by top 10% are calculated.

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2018-02-20*

## Setup

### Imports

In [2]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.io import show, output_notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import urllib as url_lib  # On Python 3.6 use "import urllib.request as url_lib".

In [3]:
tc.__version__

'0.16.1'

In [4]:
sns.set_style('white')
DPI = 300
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi']= DPI

### Create policies

Load reforms from GitHub. Policies and reforms are named according to this convention:

**`{reform}_{year}_{object}`**

For example:

* `noctc_2017_policy`: Policy without CTC using 2017 law.
* `ubi_2018_calc`: Calculator replacing CTC with UBI using 2018 (current) law.

In [7]:
# Folders where reforms live.
GITHUB_BASE_URL = 'https://raw.githubusercontent.com/'

TAXCALC_GITHUB_BASE_URL = (GITHUB_BASE_URL +
                           'open-source-economics/Tax-Calculator/master/' +
                           'taxcalc/reforms/')

def read_url(url):
    return url_lib.urlopen(url).read()

def read_reform_taxcalc_github(reform_name):
    return read_url(TAXCALC_GITHUB_BASE_URL + reform_name + '.json')

def policy_from_reform(reform):
    pol = tc.Policy()
    pol.implement_reform(reform['policy'])
    if pol.reform_errors:
        print(pol.reform_errors)
    return pol

def create_static_policy_taxcalc_github(reform_name):
    reform = tc.Calculator.read_json_param_objects(
        read_reform_taxcalc_github(reform_name), None)
    return policy_from_reform(reform)

In [8]:
y2017_policy = create_static_policy_taxcalc_github(
    '2017_law')

In [10]:
noctc_reform = {
    2018: {
        '_CTC_c': [0],
        '_DependentCredit_Child_c': [0]
    }}
noctc_2017_policy = copy.deepcopy(y2017_policy)
noctc_pol_2017.implement_reform(noctc_reform)
noctc_pol_2018 = tc.Policy()  # Can't combine with next step.
noctc_pol_2018.implement_reform(noctc_reform)

### Specify `Calculator` objects for static analyses

In [11]:
recs = tc.Records.cps_constructor()

In [12]:
def static_baseline_calc(year):
    calc = tc.Calculator(records=recs, policy=tc.Policy())
    calc.advance_to_year(year)
    calc.calc_all()
    return calc

In [18]:
def weighted_sum(df, col):
    return (df[col] * df['s006']).sum()

In [55]:
def child_ubi_reform(amount):
    return {2018: {'_UBI_u18': [amount],
                   '_UBI_ecrt': [1.0]}}

In [193]:
np.where(False, 1, np.nan)

array(nan)

In [227]:
def add_weighted_quantiles(df, col):
    df.sort_values(by=col, inplace=True)
    col_pctile = col + '_percentile_exact'
    df[col_pctile] = 100 * df['s006'].cumsum() / df['s006'].sum()
    # Integer arrays can't be NaN, so set negatives to zero.
    # TODO: Should these be null floats?
    df[col_pctile] = np.where(df[col] >= 0, df[col_pctile], 0)
    # Reduce top record, otherwise it's incorrectly rounded up.
    df[col_pctile] = np.where(df[col_pctile] >= 99.99999, 99.99999, df[col_pctile])
    df[col + '_percentile'] = np.ceil(df[col_pctile]).astype(int)
    df[col + '_decile'] = np.ceil(df[col_pctile] / 10).astype(int)
    df[col + '_quintile'] = np.ceil(df[col_pctile] / 20).astype(int)
    # TODO: Null out if negative.
    return df

In [228]:
def static_calc(use_2017_law=False, 
                ctc_treatment='keep',
                child_ubi_amount=0,
                year=2018,
                cols=['s006', 'aftertax_income', 'expanded_income', 'nu18', 'n24', 'XTOT']):
    """Creates static Calculator.

    Args:
        use_2017_law: Whether to use 2017 law vs. current law. Defaults to False.
        ctc_treatment: How the Child Tax Credit is treated. Options include:
            * 'keep': No change. Default.
            * 'repeal': End entirely.
            * 'rev_neutral_ubi': Replace with revenue-neutral child UBI.
            * 'no_cut_ubi': [NOT YET IMPLEMENTED] 
                Replace with a child UBI equal to the current maximum value.
        year: Year to advance calculations to.
        cols: Columns to extract per Calculator record. 
            Defaults to ['s006', 'expanded_income', 'aftertax_income', 'nu18', 'n24', 'XTOT'].
        
    Returns:
        DataFrame with `cols` and percentile, decile, and quintile of after-tax income.
    """
    # Initiate policy using either 2017 or current law.
    if use_2017_law:
        pol = copy.deepcopy(y2017_policy)
    else:
        pol = tc.Policy()
    # Enact reform based on ctc_treatment.
    # Repeal CTC unless it's kept.
    if ctc_treatment != 'keep':
        pol.implement_reform(noctc_reform)
    if child_ubi_amount > 0:
        pol.implement_reform(child_ubi_reform(child_ubi_amount))
    # Calculate. This is needed to calculate the revenue-neutral UBI.
    calc = tc.Calculator(records=recs, policy=pol)
    calc.advance_to_year(year)
    calc.calc_all()
    # TODO: Calculate revenue for revenue-neutral UBI.
    # Create DataFrame and add identifiers.
    df = calc.dataframe(cols)
    # Add percentiles.
    df = add_weighted_quantiles(df, 'expanded_income')
    df = add_weighted_quantiles(df, 'aftertax_income')
    # Add identifiers.
    df['use_2017_law'] = use_2017_law
    df['ctc_treatment'] = ctc_treatment
    # What's the column for the ID?
    df['id'] = df.index
    return df

In [229]:
scenarios_pre_ubi = pd.concat([
    static_calc(use_2017_law=False, ctc_treatment='keep'),
    static_calc(use_2017_law=False, ctc_treatment='repeal'),
    static_calc(use_2017_law=True, ctc_treatment='keep'),
    static_calc(use_2017_law=True, ctc_treatment='repeal')])

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [231]:
scenarios_pre_ubi.head()

,s006,aftertax_income,expanded_income,nu18,n24,XTOT,expanded_income_percentile_exact,expanded_income_percentile,expanded_income_decile,expanded_income_quintile,aftertax_income_percentile_exact,aftertax_income_percentile,aftertax_income_decile,aftertax_income_quintile,use_2017_law,ctc_treatment,id
37355,60.49,-3.555567e+06,-3.542510e+06,0.0,0.0,2.0,0.0,0,0,0,0.0,0,0,0,False,keep,37355
394742,499.17,-9.314900e+05,-9.311799e+05,1.0,1.0,3.0,0.0,0,0,0,0.0,0,0,0,False,keep,394742
29539,1209.73,-8.047257e+05,-8.047257e+05,0.0,0.0,2.0,0.0,0,0,0,0.0,0,0,0,False,keep,29539
276823,476.72,-7.817199e+05,-7.817199e+05,0.0,0.0,2.0,0.0,0,0,0,0.0,0,0,0,False,keep,276823
306673,399.35,-7.776941e+05,-7.753810e+05,0.0,0.0,2.0,0.0,0,0,0,0.0,0,0,0,False,keep,306673


Calculate child allowance.

In [134]:
aftertax_income_summary = (
    scenarios_pre_ubi.groupby(['use_2017_law', 'ctc_treatment'], as_index=False).
    apply(lambda x: 
          pd.Series({
              'aftertax_income': weighted_sum(x, 'aftertax_income'), 
              'nu18': weighted_sum(x, 'nu18')}))).reset_index()
nu18_total = aftertax_income_summary['nu18'][0]

In [135]:
aftertax_income_chg = aftertax_income_summary.pivot(
    index='use_2017_law',
    columns='ctc_treatment',
    values='aftertax_income'
)
aftertax_income_chg['chg'] = aftertax_income_chg['keep'] - aftertax_income_chg['repeal']
aftertax_income_chg['child_ubi'] = aftertax_income_chg['chg'] / nu18_total
aftertax_income_chg

ctc_treatment,keep,repeal,chg,child_ubi
use_2017_law,,,,
False,1.095007e+13,1.082995e+13,1.201240e+11,1464.524234
True,1.081362e+13,1.076178e+13,5.184284e+10,632.056117


In [136]:
scenarios = pd.concat([
    scenarios_pre_ubi,
    static_calc(use_2017_law=False, ctc_treatment='rev_neutral_ubi',
                child_ubi_amount=aftertax_income_chg.loc[False, 'child_ubi']),
    static_calc(use_2017_law=True, ctc_treatment='rev_neutral_ubi',
                child_ubi_amount=aftertax_income_chg.loc[True, 'child_ubi'])
])

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


## Analysis

* Add percentile to previous steps.
* Should there be a field for more "gross" income? Without benefits and UBI.

### Differences between `nu18` and `n24`

In [169]:
keep_cur = scenarios[(~scenarios['use_2017_law']) & 
                     (scenarios['ctc_treatment'] == 'keep')]
print ('Total children under 18: ' +
       '{:0.1f}M'.format((keep_cur['nu18'] * keep_cur['s006']).sum() / 1e6))
print ('Total children eligible for CTC: ' +
       '{:0.1f}M'.format((keep_cur['n24'] * keep_cur['s006']).sum() / 1e6))

Total children under 18: 82.0M
Total children eligible for CTC: 81.9M


How many households have nu18 > n24? n24 > nu18?

In [175]:
keep_cur.pivot_table(index='n24', columns='nu18', values='s006', aggfunc=sum)

nu18,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
n24,,,,,,,,,,,,,
0.0,1.149045e+08,5766988.03,547510.00,211161.64,69313.39,18471.38,5404.80,2750.33,50.61,3008.35,NaN,NaN,NaN
1.0,4.757063e+06,16553862.59,1788753.63,87158.45,16146.37,3154.00,218.94,NaN,NaN,NaN,NaN,NaN,NaN
2.0,2.244122e+06,854826.11,12673415.39,928240.83,47172.02,7316.58,1759.92,486.02,NaN,NaN,NaN,NaN,NaN
3.0,1.029664e+06,476427.78,28834.94,4614543.24,1584990.14,459044.89,135175.92,47639.99,15654.63,5527.24,4263.23,118.96,147.68


In [171]:
keep_cur[keep_cur['nu18'] > keep_cur['n24']]['s006'].sum() / 1e6

11.757627970000001

In [173]:
keep_cur[keep_cur['n24'] > keep_cur['nu18']]['s006'].sum() / 1e6

9.3909378300000022

### Distributional impact

In [137]:
scenarios.loc[1000, :].sort_values('use_2017_law')

,s006,aftertax_income,expanded_income,nu18,n24,use_2017_law,ctc_treatment,id
1000,14.39,80743.440366,75425.491384,3.0,2.0,False,keep,1000
1000,14.39,78513.210786,75425.491384,3.0,2.0,False,repeal,1000
1000,14.39,82906.783490,79819.064088,3.0,2.0,False,rev_neutral_ubi,1000
1000,14.39,80527.250786,75425.491384,3.0,2.0,True,keep,1000
1000,14.39,78527.250786,75425.491384,3.0,2.0,True,repeal,1000
1000,14.39,80423.419136,77321.659734,3.0,2.0,True,rev_neutral_ubi,1000


In [159]:
scenario_pivot = scenarios.pivot_table(values=['expanded_income', 'aftertax_income'],
                                       index=['id', 'use_2017_law', 's006', 'nu18', 'n24'],
                                       columns='ctc_treatment').reset_index()
# Adapted from https://stackoverflow.com/q/42099024/1840471.
scenario_pivot.columns = ["_".join((j, i)) for i, j in scenario_pivot.columns]
scenario_pivot.columns = scenario_pivot.columns.str.lstrip('_')

In [160]:
scenario_pivot['afti_chg_ctc'] = (
    (scenario_pivot['repeal_aftertax_income'] - scenario_pivot['keep_aftertax_income']) / 
    scenario_pivot['keep_aftertax_income'])
scenario_pivot['afti_chg_ubi'] = (
    (scenario_pivot['rev_neutral_ubi_aftertax_income'] - scenario_pivot['keep_aftertax_income']) / 
    scenario_pivot['keep_aftertax_income'])
scenario18 = scenario_pivot[~scenario_pivot['use_2017_law']].drop(columns='use_2017_law')

In [184]:
scenario18[(scenario18['rev_neutral_ubi_aftertax_income'] >= 0) &
           (scenario18['keep_aftertax_income'] >= 0) &
           (scenario18['afti_chg_ubi'] < np.inf)].sort_values('afti_chg_ubi').tail()

,id,s006,nu18,n24,keep_aftertax_income,repeal_aftertax_income,rev_neutral_ubi_aftertax_income,keep_expanded_income,repeal_expanded_income,rev_neutral_ubi_expanded_income,afti_chg_ctc,afti_chg_ubi
321318,160659,824.31,2.0,1.0,1.739164,1.739164,2930.787633,1.739164,1.739164,2930.787633,0.0,1684.170422
318758,159379,457.08,2.0,1.0,1.739164,1.739164,2930.787633,1.739164,1.739164,2930.787633,0.0,1684.170422
413628,206814,1007.05,3.0,2.0,1.549691,1.549691,4395.122394,1.549691,1.549691,4395.122394,0.0,2835.128587
258024,129012,203.45,3.0,2.0,1.541713,1.541713,4395.114416,1.541713,1.541713,4395.114416,0.0,2849.799369
450262,225131,355.67,3.0,0.0,1.363286,1.363286,4394.935990,1.363286,1.363286,4394.935990,0.0,3222.781213
